In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
from typing import Union, Optional, Dict

In [ ]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
seed = 69
seed_everything(seed=seed)

In [ ]:
class Node:
    def __init__(self, reward:Union[int, float], children:Optional[Dict[str, float]]=None):
        self.reward = reward
        self.terminal = children is None
        self.children = children

    def transition(self):
        output_name = None
        if self.terminal: return output_name
        roll = random.random()
        total = 0
        for node in self.children.keys():
            output_name = node
            total += self.children[node]
            if total >= roll: return output_name
        return output_name

In [ ]:
class CavesPaths:
    def __init__(self, default_paths:Optional[Dict[str,float]]=None, node_params:Optional[dict]=None):
        if default_paths is None: 
            default_paths = {
                'A': .5,
                'B': .3,
                'C': .2
            }
        if node_params is None:
            node_params = {
                'start': (0, default_paths),
                'A': (2, default_paths),
                'B': (1, default_paths),
                'C': (0, None)
            }

        self.default_paths = default_paths
        self.node_params = node_params

    def build_node(self, name:str) -> Node:
        return Node(*self.node_params[name])

    def build_path(self):
        node = self.build_node('start')
        distance = node.reward
        paths = []
        while node is not None:
            next_node_name = node.transition()
            if next_node_name is None: break
            paths.append(next_node_name)
            node = self.build_node(next_node_name)
            distance += node.reward
        return paths, distance

In [ ]:
caves_paths = CavesPaths()

In [ ]:
for _ in range(5):
    path, distance = caves_paths.build_path()
    print(f'distance traveled: {distance}\n\tpath: {path}')

distance traveled: 8
	path: ['B', 'A', 'A', 'B', 'A', 'C']
distance traveled: 1
	path: ['B', 'C']
distance traveled: 2
	path: ['A', 'C']
distance traveled: 1
	path: ['B', 'C']
distance traveled: 21
	path: ['A', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C']


In [ ]:
# we want to know the probability that AAB appears in the sequence and 
# it appears before any BAA appears
def check_aab(path):
    assert type(path) == list
    if len(path) < 4: return False
    for i in range(len(path) - 3):
        phrase = path[i] + path[i+1] + path[i+2]
        if phrase == 'AAB' : return True
        if phrase == 'BAA' : return False
    return False

In [ ]:
def create_data(m:int=10000000) -> pd.DataFrame:
    paths = []
    distances = []

    for _ in range(m):
        path, distance = caves_paths.build_path()
        paths.append(path)
        distances.append(distance)

    paths = pd.Series(paths)
    distances = pd.Series(distances, dtype=int)

    df = pd.DataFrame({
        'paths':paths,
        'days':distances
    })

    targets = []
    for i in range(m):
        has_aab = 1 if check_aab(paths[i]) else 0
        targets.append(has_aab)

    targets = pd.Series(targets, dtype=int)
    df['target'] = targets

    return df

In [ ]:
df = create_data()

In [ ]:
df.head()

In [ ]:
df['target'].mean()

In [ ]:
X = df.copy()
y = X.pop('target')

In [ ]:
def make_random_sample(y):
    m = len(y)
    step = int(m / 100)
    splits = range(0, m, step)
    means = []
    for idx in splits:
        values = y[idx:idx + step]
        mean = values.mean()
        means.append(mean)
    
    means = pd.Series(means, dtype=np.float)
    means -= means.mean()
    means = pd.value_counts(means, bins=100)
    return means

In [ ]:
samples = make_random_sample(y)

In [ ]:
sns.histplot(samples)